In [9]:
import openreview
import pandas as pd
import datetime as date
import tqdm
import json
import ast
import itertools
import matplotlib.pyplot as plt
import requests, json
client = openreview.Client(baseurl='https://api.openreview.net')

In [54]:
df_author_profiles = pd.read_csv('data/authors/df_author_profiles.csv')
df_submissions = pd.read_csv('data/submissions/df_submissions.csv',index_col=0)

In [55]:
def lambda_get_first_last_author(row):
    first_author_name = str()
    first_author_id = str()
    last_author_name = str()
    last_author_id = str()
    
    try:
        lst_names = ast.literal_eval(row['authors'])
        lst_ids = ast.literal_eval(row['authorids'])
        first_author_name = lst_names[0]
        last_author_name = lst_names[-1]
        first_author_id = lst_ids[0]
        last_author_id = lst_ids[-1]
    except:
        first_author_name = 'error'
        last_author_name = 'error'
        first_author_id = 'error'
        last_author_id = 'error'
        
    return first_author_name, last_author_name, first_author_id, last_author_id

In [56]:


df_submissions[['first_author_name','last_author_name','first_author_id','last_author_id']] = df_submissions.apply(lambda x: lambda_get_first_last_author(x), axis = 1 , result_type='expand')

In [57]:
df_submissions

,id,original,number,cdate,mdate,tcdate,tmdate,ddate,forum,referent,...,venueid,_bibtex,paperhash,author_emails,code,data,first_author_name,last_author_name,first_author_id,last_author_id
0,S1VaB4cex,NaN,214,NaN,NaN,1478276540541,1658433096672,NaN,S1VaB4cex,NaN,...,ICLR.cc/2017/conference,"@inproceedings{\nlarsson2017fractalnet,\ntitle...",larsson|fractalnet_ultradeep_neural_networks_w...,NaN,NaN,NaN,Gustav Larsson,Gregory Shakhnarovich,larsson@cs.uchicago.edu,greg@ttic.edu
1,H1W1UN9gg,NaN,215,NaN,NaN,1478276569300,1658433096913,NaN,H1W1UN9gg,NaN,...,ICLR.cc/2017/conference,"@inproceedings{\nschoenholz2017deep,\ntitle={D...",schoenholz|deep_information_propagation,NaN,NaN,NaN,Samuel S. Schoenholz,Jascha Sohl-Dickstein,schsam@google.com,jaschasd@google.com
2,r1GKzP5xx,NaN,370,NaN,NaN,1478287994558,1658434077890,NaN,r1GKzP5xx,NaN,...,ICLR.cc/2017/conference,"@misc{\nlee2017making,\ntitle={Making Stochast...",laurent|recurrent_normalization_propagation,NaN,NaN,NaN,César Laurent,Pascal Vincent,cesar.laurent@umontreal.ca,pascal.vincent@umontreal.ca
3,S1J0E-71l,NaN,2,NaN,NaN,1476756679535,1658433097410,NaN,S1J0E-71l,NaN,...,ICLR.cc/2017/conference,"@misc{\nrocki2017surprisaldriven,\ntitle={Surp...",k|surprisaldriven_feedback_in_recurrent_networks,kmrocki@us.ibm.com,NaN,NaN,error,error,error,error
4,SJGCiw5gl,NaN,427,NaN,NaN,1478290378652,1658433097637,NaN,SJGCiw5gl,NaN,...,ICLR.cc/2017/conference,"@inproceedings{\nmolchanov2017pruning,\ntitle=...",molchanov|pruning_convolutional_neural_network...,NaN,NaN,NaN,Pavlo Molchanov,Jan Kautz,pmolchanov@nvidia.com,jkautz@nvidia.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,BkE8NjCqYm,Hyxdt-7YYm,6,1.538088e+12,NaN,1538087726374,1545355398839,NaN,BkE8NjCqYm,NaN,...,NaN,"@misc{\ncohen2019unconstrained,\ntitle={(Uncon...",cohen|unconstrained_beam_search_is_sensitive_t...,NaN,NaN,NaN,Eldan Cohen,J. Christopher Beck,ecohen@mie.utoronto.ca,jcb@mie.utoronto.ca
1415,BJfIVjAcKm,SJl8QZNFKm,4,1.538088e+12,NaN,1538087725988,1644492706715,NaN,BJfIVjAcKm,NaN,...,NaN,"@inproceedings{\nxiao2018training,\ntitle={Tra...",xiao|training_for_faster_adversarial_robustnes...,NaN,[![github](/images/github_icon.svg) MadryLab/r...,NaN,Kai Y. Xiao,Aleksander Madry,kaix@mit.edu,madry@mit.edu
1416,Hkl84iCcFm,HygqaYAcFm,3,1.538088e+12,NaN,1538087725796,1545355412821,NaN,Hkl84iCcFm,NaN,...,NaN,"@misc{\nlagzi2019residual,\ntitle={{RESIDUAL} ...",lagzi|residual_networks_classify_inputs_based_...,NaN,NaN,NaN,Fereshteh Lagzi,Fereshteh Lagzi,lagzi@informatik.uni-freiburg.de,lagzi@informatik.uni-freiburg.de
1417,S1gUVjCqKm,HyenhvCqKX,2,1.538088e+12,NaN,1538087725593,1545355420919,NaN,S1gUVjCqKm,NaN,...,NaN,"@misc{\nhan2019unsupervised,\ntitle={Unsupervi...",han|unsupervised_classification_into_unknown_n...,NaN,NaN,NaN,Sungyeob Han,Jungwoo Lee,syhan@cml.snu.ac.kr,junglee@snu.ac.kr


In [7]:
search = 'https://api.openalex.org/authors?filter=display_name.search:'
name = 'Liu Yang'

test = search + name

In [59]:
def lambda_get_OpenAlex_request(row):
    search = 'https://api.openalex.org/authors?filter=display_name.search:'
    first_author_name = row['first_author_name']
    last_author_name = row['last_author_name']
    
    search_first_author = search + first_author_name
    return json.loads(requests.get(search_first_author).text)['results']

In [62]:
tqdm.tqdm.pandas()


df_submissions.progress_apply(lambda x : lambda_get_OpenAlex_request(x), axis = 1)

100%|███████████████████████████████████████| 2820/2820 [23:50<00:00,  1.97it/s]


0       [{'id': 'https://openalex.org/A2338325017', 'o...
1       [{'id': 'https://openalex.org/A2833214316', 'o...
2       [{'id': 'https://openalex.org/A2250018335', 'o...
3       [{'id': 'https://openalex.org/A401312156', 'or...
4       [{'id': 'https://openalex.org/A2777799004', 'o...
                              ...                        
1414    [{'id': 'https://openalex.org/A2604423490', 'o...
1415                                                   []
1416    [{'id': 'https://openalex.org/A1974555589', 'o...
1417    [{'id': 'https://openalex.org/A2613955392', 'o...
1418    [{'id': 'https://openalex.org/A2805077430', 'o...
Length: 2820, dtype: object

In [63]:
df_submissions

,id,original,number,cdate,mdate,tcdate,tmdate,ddate,forum,referent,...,venueid,_bibtex,paperhash,author_emails,code,data,first_author_name,last_author_name,first_author_id,last_author_id
0,S1VaB4cex,NaN,214,NaN,NaN,1478276540541,1658433096672,NaN,S1VaB4cex,NaN,...,ICLR.cc/2017/conference,"@inproceedings{\nlarsson2017fractalnet,\ntitle...",larsson|fractalnet_ultradeep_neural_networks_w...,NaN,NaN,NaN,Gustav Larsson,Gregory Shakhnarovich,larsson@cs.uchicago.edu,greg@ttic.edu
1,H1W1UN9gg,NaN,215,NaN,NaN,1478276569300,1658433096913,NaN,H1W1UN9gg,NaN,...,ICLR.cc/2017/conference,"@inproceedings{\nschoenholz2017deep,\ntitle={D...",schoenholz|deep_information_propagation,NaN,NaN,NaN,Samuel S. Schoenholz,Jascha Sohl-Dickstein,schsam@google.com,jaschasd@google.com
2,r1GKzP5xx,NaN,370,NaN,NaN,1478287994558,1658434077890,NaN,r1GKzP5xx,NaN,...,ICLR.cc/2017/conference,"@misc{\nlee2017making,\ntitle={Making Stochast...",laurent|recurrent_normalization_propagation,NaN,NaN,NaN,César Laurent,Pascal Vincent,cesar.laurent@umontreal.ca,pascal.vincent@umontreal.ca
3,S1J0E-71l,NaN,2,NaN,NaN,1476756679535,1658433097410,NaN,S1J0E-71l,NaN,...,ICLR.cc/2017/conference,"@misc{\nrocki2017surprisaldriven,\ntitle={Surp...",k|surprisaldriven_feedback_in_recurrent_networks,kmrocki@us.ibm.com,NaN,NaN,error,error,error,error
4,SJGCiw5gl,NaN,427,NaN,NaN,1478290378652,1658433097637,NaN,SJGCiw5gl,NaN,...,ICLR.cc/2017/conference,"@inproceedings{\nmolchanov2017pruning,\ntitle=...",molchanov|pruning_convolutional_neural_network...,NaN,NaN,NaN,Pavlo Molchanov,Jan Kautz,pmolchanov@nvidia.com,jkautz@nvidia.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1414,BkE8NjCqYm,Hyxdt-7YYm,6,1.538088e+12,NaN,1538087726374,1545355398839,NaN,BkE8NjCqYm,NaN,...,NaN,"@misc{\ncohen2019unconstrained,\ntitle={(Uncon...",cohen|unconstrained_beam_search_is_sensitive_t...,NaN,NaN,NaN,Eldan Cohen,J. Christopher Beck,ecohen@mie.utoronto.ca,jcb@mie.utoronto.ca
1415,BJfIVjAcKm,SJl8QZNFKm,4,1.538088e+12,NaN,1538087725988,1644492706715,NaN,BJfIVjAcKm,NaN,...,NaN,"@inproceedings{\nxiao2018training,\ntitle={Tra...",xiao|training_for_faster_adversarial_robustnes...,NaN,[![github](/images/github_icon.svg) MadryLab/r...,NaN,Kai Y. Xiao,Aleksander Madry,kaix@mit.edu,madry@mit.edu
1416,Hkl84iCcFm,HygqaYAcFm,3,1.538088e+12,NaN,1538087725796,1545355412821,NaN,Hkl84iCcFm,NaN,...,NaN,"@misc{\nlagzi2019residual,\ntitle={{RESIDUAL} ...",lagzi|residual_networks_classify_inputs_based_...,NaN,NaN,NaN,Fereshteh Lagzi,Fereshteh Lagzi,lagzi@informatik.uni-freiburg.de,lagzi@informatik.uni-freiburg.de
1417,S1gUVjCqKm,HyenhvCqKX,2,1.538088e+12,NaN,1538087725593,1545355420919,NaN,S1gUVjCqKm,NaN,...,NaN,"@misc{\nhan2019unsupervised,\ntitle={Unsupervi...",han|unsupervised_classification_into_unknown_n...,NaN,NaN,NaN,Sungyeob Han,Jungwoo Lee,syhan@cml.snu.ac.kr,junglee@snu.ac.kr


In [10]:
response = requests.get(test)
response 

<Response [200]>

In [23]:
res = json.loads(response.text)



In [32]:
res['results']

[{'id': 'https://openalex.org/A2776297621',
  'orcid': 'https://orcid.org/0000-0001-5477-2186',
  'display_name': 'Yang Liu',
  'display_name_alternatives': ['刘阳'],
  'relevance_score': 3901.0784,
  'works_count': 2764,
  'cited_by_count': 124390,
  'ids': {'openalex': 'https://openalex.org/A2776297621',
   'orcid': 'https://orcid.org/0000-0001-5477-2186',
   'mag': '2776297621'},
  'last_known_institution': {'id': 'https://openalex.org/I157773358',
   'ror': 'https://ror.org/0064kty71',
   'display_name': 'Sun Yat-sen University',
   'country_code': 'CN',
   'type': 'education'},
  'x_concepts': [{'id': 'https://openalex.org/C86803240',
    'wikidata': 'https://www.wikidata.org/wiki/Q420',
    'display_name': 'Biology',
    'level': 0,
    'score': 77.9},
   {'id': 'https://openalex.org/C185592680',
    'wikidata': 'https://www.wikidata.org/wiki/Q2329',
    'display_name': 'Chemistry',
    'level': 0,
    'score': 76.3},
   {'id': 'https://openalex.org/C121332964',
    'wikidata': 'ht